## import library

In [1]:
import os
import numpy as np
import tensorflow as tf
from data_loader import *
from data_utils import *
from model import *
from const import *

c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\shimd\anaconda3\envs\mondeique\lib\site-packages\ten

# prepare data

In [2]:
train_data, test_data = getBagImage()

Reading csv................
Load bag image...................
Number of bag train data:  1941
Number of bag test data:  530
---------------------------------------------------------------


In [3]:
def one_hot(index, num_classes):
    tmp = np.zeros(num_classes, dtype=np.float32)
    tmp[index] = 1.0
    return tmp

# Main Training code

## Input tensor + Model Network 

In [4]:
sess = tf.InteractiveSession()
global_step = tf.contrib.framework.get_or_create_global_step()

x, y, mask = input_tensor()

y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_carrying_conv, y_charm_conv, y_decoration_conv, is_training, keep_prob = multi_label_net(x)

W0819 14:42:10.581415  5436 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0819 14:42:10.582413  5436 deprecation.py:323] From <ipython-input-4-eaa24b9c7f9c>:2: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0819 14:42:10.587399  5436 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:7: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 14:42:10.591361  5436 deprecation_wrapper.py:119] From C:\Users\shimd\

## Loss + mask 씌우기

In [5]:
color_loss, shape_loss, opening_loss, strap_loss, pattern_loss, carrying_loss, charm_loss, decoration_loss, l2_loss, total_loss = selective_loss(y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_carrying_conv, y_charm_conv, y_decoration_conv,y, mask)

train_step = train_op(total_loss, global_step)

color_mask = tf.get_collection('color_mask')[0]
shape_mask = tf.get_collection('shape_mask')[0]
opening_mask = tf.get_collection('opening_mask')[0]
strap_mask = tf.get_collection('strap_mask')[0]
pattern_mask = tf.get_collection('pattern_mask')[0]
carrying_mask = tf.get_collection('carrying_mask')[0]
charm_mask = tf.get_collection('charm_mask')[0]
decoration_mask = tf.get_collection('decoration_mask')[0]

y_color = tf.get_collection('y_color')[0]
y_shape = tf.get_collection('y_shape')[0]
y_opening = tf.get_collection('y_opening')[0]
y_strap = tf.get_collection('y_strap')[0]
y_pattern = tf.get_collection('y_pattern')[0]
y_carrying = tf.get_collection('y_carrying')[0]
y_charm = tf.get_collection('y_charm')[0]
y_decoration = tf.get_collection('y_decoration')[0]

color_correct_prediction = tf.equal(tf.argmax(y_color_conv, 1), tf.argmax(y_color, 1))
shape_correct_prediction = tf.equal(tf.argmax(y_shape_conv, 1), tf.argmax(y_shape, 1))
opening_correct_prediction = tf.equal(tf.argmax(y_opening_conv, 1), tf.argmax(y_opening, 1))
strap_correct_prediction = tf.equal(tf.argmax(y_strap_conv, 1), tf.argmax(y_strap, 1))
pattern_correct_prediction = tf.equal(tf.argmax(y_pattern_conv, 1), tf.argmax(y_pattern, 1))
carrying_correct_prediction = tf.equal(tf.argmax(y_carrying_conv, 1), tf.argmax(y_carrying, 1))
charm_correct_prediction = tf.equal(tf.argmax(y_charm_conv, 1), tf.argmax(y_charm, 1))
decoration_correct_prediction = tf.equal(tf.argmax(y_decoration_conv, 1), tf.argmax(y_decoration, 1))

color_true_pred = tf.reduce_sum(tf.cast(color_correct_prediction, dtype=tf.float32) * color_mask)
shape_true_pred = tf.reduce_sum(tf.cast(shape_correct_prediction, dtype=tf.float32) * shape_mask)
opening_true_pred = tf.reduce_sum(tf.cast(opening_correct_prediction, dtype=tf.float32) * opening_mask)
strap_true_pred = tf.reduce_sum(tf.cast(strap_correct_prediction, dtype=tf.float32) * strap_mask)
pattern_true_pred = tf.reduce_sum(tf.cast(pattern_correct_prediction, dtype=tf.float32) * pattern_mask)
carrying_true_pred = tf.reduce_sum(tf.cast(carrying_correct_prediction, dtype=tf.float32) * carrying_mask)
charm_true_pred = tf.reduce_sum(tf.cast(charm_correct_prediction, dtype=tf.float32) * charm_mask)
decoration_true_pred = tf.reduce_sum(tf.cast(decoration_correct_prediction, dtype=tf.float32) * decoration_mask)

real_train_data = []

# Mask : color -> 0 , shape -> 1, opening -> 2, strap -> 3, pattern -> 4, carrying -> 5, charm -> 6, decoration -> 7
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][0], 16), 0.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][1], 16), 1.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][2], 16), 2.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][3], 16), 3.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][4], 16), 4.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][5], 16), 5.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][6], 16), 6.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][7], 16), 7.0))

saver = tf.train.Saver()

W0819 14:42:11.330408  5436 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:94: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

W0819 14:42:11.341356  5436 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:122: The name tf.log is deprecated. Please use tf.math.log instead.

W0819 14:42:11.409174  5436 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:158: The name tf.train.exponential_decay is deprecated. Please use tf.compat.v1.train.exponential_decay instead.

W0819 14:42:11.414161  5436 deprecation_wrapper.py:119] From C:\Users\shimd\Desktop\multi-label-classification\model.py:159: The name tf.train.MomentumOptimizer is deprecated. Please use tf.compat.v1.train.MomentumOptimizer instead.



## restore model 생성

In [6]:
if not os.path.isfile(SAVE_FOLDER3 + 'model_v3_3.ckpt.index'):
    print('Create new model')
    sess.run(tf.global_variables_initializer())
    print('OK')
else:
    print('Restoring existed model')
    saver.restore(sess, SAVE_FOLDER3 + 'model_v3_3.ckpt')
    print('OK')

loss_summary_placeholder = tf.placeholder(tf.float32)
tf.summary.scalar('loss', loss_summary_placeholder)
merge_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./summary/")

learning_rate = tf.get_collection('learning_rate')[0]

Create new model
OK


## epoch 에 따라서 training 수행

In [7]:
current_epoch = (int)(global_step.eval() / (len(real_train_data) // BATCH_SIZE))
for epoch in range(current_epoch + 1, NUM_EPOCHS):
    print('Epoch:', str(epoch))
    np.random.shuffle(real_train_data)
    train_img = []
    train_label = []
    train_mask = []

    for i in range(len(real_train_data)):
        train_img.append(real_train_data[i][0])
        train_label.append(real_train_data[i][1])
        train_mask.append(real_train_data[i][2])

    number_batch = len(real_train_data) // BATCH_SIZE

    avg_ttl = []
    avg_rgl = []
    avg_color_loss = []
    avg_shape_loss = []
    avg_opening_loss = []
    avg_strap_loss = []
    avg_pattern_loss = []
    avg_carrying_loss = []
    avg_charm_loss = []
    avg_decoration_loss = []

    color_nb_true_pred = 0
    shape_nb_true_pred = 0
    opening_nb_true_pred = 0
    strap_nb_true_pred = 0
    pattern_nb_true_pred = 0
    carrying_nb_true_pred = 0
    charm_nb_true_pred = 0
    decoration_nb_true_pred = 0

    color_nb_train = 0
    shape_nb_train = 0
    opening_nb_train = 0
    strap_nb_train = 0
    pattern_nb_train = 0
    carrying_nb_train = 0
    charm_nb_train = 0
    decoration_nb_train = 0

    print("Learning rate: %f" % learning_rate.eval())
    for batch in range(number_batch):
        # print('Training on batch {0}/{1}'.format(str(batch + 1), str(number_batch)))
        top = batch * BATCH_SIZE
        bot = min((batch + 1) * BATCH_SIZE, len(real_train_data))
        batch_img = np.asarray(train_img[top:bot])
        batch_label = np.asarray(train_label[top:bot])
        batch_mask = np.asarray(train_mask[top:bot])
        

        for i in range(BATCH_SIZE):
            if batch_mask[i] == 0.0:
                color_nb_train += 1
            else:
                if batch_mask[i] == 1.0:
                    shape_nb_train += 1
                else:
                    if batch_mask[i] == 2.0:
                        opening_nb_train += 1
                    else:
                        if batch_mask[i] == 3.0:
                            strap_nb_train += 1
                        else:
                            if batch_mask[i] == 4.0:
                                pattern_nb_train += 1
                            else:
                                if batch_mask[i] == 5.0:
                                    carrying_nb_train += 1
                                else:
                                    if batch_mask[i] == 6.0:
                                        charm_nb_train += 1
                                    else:
                                        decoration_nb_train +=1

        batch_img = augmentation(batch_img, 64)

        ttl, colorl, shapel, openingl, strapl, patternl, carryingl, charml, decol, l2l, _ = sess.run([total_loss, color_loss, shape_loss, opening_loss, strap_loss,
                                                                                      pattern_loss, carrying_loss, charm_loss, decoration_loss, l2_loss, train_step],
                                                                                      feed_dict={x: batch_img, y: batch_label, mask: batch_mask, is_training: True, keep_prob: DROP_OUT_PROB})

        color_nb_true_pred += sess.run(color_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True,keep_prob: DROP_OUT_PROB})

        shape_nb_true_pred += sess.run(shape_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True, keep_prob: DROP_OUT_PROB})

        opening_nb_true_pred += sess.run(opening_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                       is_training: True, keep_prob: DROP_OUT_PROB})

        strap_nb_true_pred += sess.run(strap_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: True, keep_prob: DROP_OUT_PROB})

        pattern_nb_true_pred += sess.run(pattern_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                       is_training: True, keep_prob: DROP_OUT_PROB})

        carrying_nb_true_pred += sess.run(carrying_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob: DROP_OUT_PROB})

        charm_nb_true_pred += sess.run(charm_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob:DROP_OUT_PROB})

        decoration_nb_true_pred += sess.run(decoration_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                         is_training: True, keep_prob: DROP_OUT_PROB})

        avg_ttl.append(ttl)
        avg_color_loss.append(colorl)
        avg_shape_loss.append(shapel)
        avg_opening_loss.append(openingl)
        avg_strap_loss.append(strapl)
        avg_pattern_loss.append(patternl)
        avg_carrying_loss.append(carryingl)
        avg_charm_loss.append(charml)
        avg_decoration_loss.append(decol)

        avg_rgl.append(l2l)

    color_train_accuracy = color_nb_true_pred * 1.0 / color_nb_train
    shape_train_accuracy = shape_nb_true_pred * 1.0 / shape_nb_train
    opening_train_accuracy = opening_nb_true_pred * 1.0 / opening_nb_train
    strap_train_accuracy = strap_nb_true_pred * 1.0 / strap_nb_train
    pattern_train_accuracy = pattern_nb_true_pred * 1.0 / pattern_nb_train
    carrying_train_accuracy = carrying_nb_true_pred * 1.0 / carrying_nb_train
    charm_train_accuracy = charm_nb_true_pred * 1.0 / charm_nb_train
    decoration_train_accuracy = decoration_nb_true_pred * 1.0 / decoration_nb_train

    avg_color_loss = np.average(avg_color_loss)
    avg_shape_loss = np.average(avg_shape_loss)
    avg_opening_loss = np.average(avg_opening_loss)
    avg_strap_loss = np.average(avg_strap_loss)
    avg_pattern_loss = np.average(avg_pattern_loss)
    avg_carrying_loss = np.average(avg_carrying_loss)
    avg_charm_loss = np.average(avg_charm_loss)
    avg_decoration_loss = np.average(avg_decoration_loss)

    avg_rgl = np.average(avg_rgl)
    avg_ttl = np.average(avg_ttl)

    #     print('Avg_ttl: ' + str(avg_ttl))
    #     print('loss_summary_placeholder: ' + str(loss_summary_placeholder))
    #     print('merge_summary: ' + str(merge_summary))
    
    # 밑의 두 줄 error 발생해서 주석처리 해놓았음. TODO!
    
    #summary = sess.run(merge_summary, feed_dict={loss_summary_placeholder: avg_ttl})
    #writer.add_summary(summary, global_step=epoch)

    with open('log.csv', 'w+') as f:
        # epochs, color_train_accuracy, shape_train_accuracy, opening_train_accuracy,
        # avg_color_loss, avg_shape_loss, avg_opening_loss, avg_ttl, avg_rgl
        f.write('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14}, {15}, {16}, {17}, {18}\n'.format(current_epoch, color_train_accuracy, shape_train_accuracy, opening_train_accuracy,
                                                                                            strap_train_accuracy, pattern_train_accuracy, carrying_train_accuracy, charm_train_accuracy, decoration_train_accuracy,
                                                                                            avg_color_loss, avg_shape_loss, avg_opening_loss, avg_strap_loss, avg_pattern_loss,
                                                                                            avg_carrying_loss, avg_charm_loss, avg_decoration_loss, avg_ttl, avg_rgl))

    print('color task train accuracy: ' + str(color_train_accuracy * 100))
    print('shape task train accuracy: ' + str(shape_train_accuracy * 100))
    print('opening task train accuracy: ' + str(opening_train_accuracy * 100))
    print('strap task train accuracy: ' + str(strap_train_accuracy * 100))
    print('pattern task train accuracy: ' + str(pattern_train_accuracy * 100))
    print('carrying task train accuracy: ' + str(carrying_train_accuracy * 100))
    print('charm task train accuracy: ' + str(charm_train_accuracy * 100))
    print('decoration task train accuracy: ' + str(decoration_train_accuracy * 100))
    
    print('\n')

    print('Total loss: ' + str(avg_ttl) + '. L2-loss: ' + str(avg_rgl))
    print('Color loss: ' + str(avg_color_loss))
    print('Shape loss: ' + str(avg_shape_loss))
    print('opening loss: ' + str(avg_opening_loss))
    print('strap loss: ' + str(avg_strap_loss))
    print('pattern loss: ' + str(avg_pattern_loss))
    print('carrying loss: ' + str(avg_carrying_loss))
    print('charm loss: ' + str(avg_charm_loss))
    print('decoration loss: ' + str(avg_decoration_loss))

    print('\n')

    saver.save(sess, SAVE_FOLDER3 + 'model_v3_3.ckpt')

Epoch: 1
Learning rate: 0.010000
color task train accuracy: 28.969072164948457
shape task train accuracy: 63.38318720990201
opening task train accuracy: 33.91752577319588
strap task train accuracy: 66.03092783505154
pattern task train accuracy: 70.77319587628867
carrying task train accuracy: 40.44307058217414
charm task train accuracy: 41.06130860381247
decoration task train accuracy: 84.37338834450749


Total loss: 11.775662. L2-loss: 3.9789321
Color loss: 1.9672409
Shape loss: 1.1841718
opening loss: 0.009122978
strap loss: 0.76855177
pattern loss: 0.3715088
carrying loss: 1.3548506
charm loss: 1.3523319
decoration loss: 0.78895086


Epoch: 2
Learning rate: 0.010000
color task train accuracy: 30.0875837197321
shape task train accuracy: 63.96907216494845
opening task train accuracy: 33.883445074780816
strap task train accuracy: 66.61514683153014
pattern task train accuracy: 81.71045852653272
carrying task train accuracy: 45.77319587628866
charm task train accuracy: 44.4043321299639
de

Epoch: 14
Learning rate: 0.008574
color task train accuracy: 37.24884080370943
shape task train accuracy: 66.04842864502834
opening task train accuracy: 33.8659793814433
strap task train accuracy: 67.69706336939721
pattern task train accuracy: 77.3078906652914
carrying task train accuracy: 62.68041237113402
charm task train accuracy: 62.970603403816405
decoration task train accuracy: 84.37338834450749


Total loss: 6.583521. L2-loss: 0.62414676
Color loss: 1.6384975
Shape loss: 0.94774306
opening loss: 0.0018001783
strap loss: 0.5544177
pattern loss: 0.32891858
carrying loss: 0.94330466
charm loss: 0.94600445
decoration loss: 0.5986879


Epoch: 15
Learning rate: 0.008574
color task train accuracy: 36.907216494845365
shape task train accuracy: 65.97938144329896
opening task train accuracy: 33.81443298969072
strap task train accuracy: 67.62886597938144
pattern task train accuracy: 78.82534775888718
carrying task train accuracy: 63.31269349845201
charm task train accuracy: 63.111798042246

Epoch: 27
Learning rate: 0.007351
color task train accuracy: 42.87925696594427
shape task train accuracy: 70.2730551262236
opening task train accuracy: 33.84853168469861
strap task train accuracy: 70.17001545595053
pattern task train accuracy: 83.08406395048993
carrying task train accuracy: 66.8041237113402
charm task train accuracy: 66.39175257731958
decoration task train accuracy: 84.27835051546391


Total loss: 6.452057. L2-loss: 0.8343973
Color loss: 1.5188115
Shape loss: 0.91297966
opening loss: 0.001064512
strap loss: 0.49162138
pattern loss: 0.3165214
carrying loss: 0.8781191
charm loss: 0.8759377
decoration loss: 0.62260455


Epoch: 28
Learning rate: 0.007351
color task train accuracy: 42.16494845360825
shape task train accuracy: 69.3298969072165
opening task train accuracy: 33.8659793814433
strap task train accuracy: 68.98505924781041
pattern task train accuracy: 75.55440948942754
carrying task train accuracy: 66.54639175257732
charm task train accuracy: 66.838576585869
decora

Epoch: 40
Learning rate: 0.006302
color task train accuracy: 50.05154639175258
shape task train accuracy: 71.5610510046368
opening task train accuracy: 33.8659793814433
strap task train accuracy: 70.87628865979381
pattern task train accuracy: 76.90721649484537
carrying task train accuracy: 70.0876740587932
charm task train accuracy: 70.53065430190624
decoration task train accuracy: 84.32181536874678


Total loss: 6.2562385. L2-loss: 1.0406748
Color loss: 1.2828337
Shape loss: 0.8637419
opening loss: 0.000670159
strap loss: 0.49724513
pattern loss: 0.31081706
carrying loss: 0.84358615
charm loss: 0.840715
decoration loss: 0.5759544


Epoch: 41
Learning rate: 0.006302
color task train accuracy: 53.63214837712519
shape task train accuracy: 71.66409067490984
opening task train accuracy: 33.8659793814433
strap task train accuracy: 71.46542827657379
pattern task train accuracy: 74.12371134020619
carrying task train accuracy: 69.69072164948453
charm task train accuracy: 69.22680412371133
deco

Epoch: 53
Learning rate: 0.005404
color task train accuracy: 55.876288659793815
shape task train accuracy: 72.6944873776404
opening task train accuracy: 33.79772961816305
strap task train accuracy: 72.57731958762886
pattern task train accuracy: 75.19339865910263
carrying task train accuracy: 70.17001545595053
charm task train accuracy: 69.9124162802679
decoration task train accuracy: 84.3298969072165


Total loss: 6.2574053. L2-loss: 1.1978014
Color loss: 1.1850173
Shape loss: 0.8351839
opening loss: 0.0005370058
strap loss: 0.46413565
pattern loss: 0.30174223
carrying loss: 0.85524607
charm loss: 0.8376207
decoration loss: 0.58012116


Epoch: 54
Learning rate: 0.005404
color task train accuracy: 56.162970603403814
shape task train accuracy: 73.24742268041237
opening task train accuracy: 33.84853168469861
strap task train accuracy: 72.15059308922125
pattern task train accuracy: 75.9278350515464
carrying task train accuracy: 71.54639175257732
charm task train accuracy: 71.09737248840804

...

Epoch: 235
Learning rate: 0.000566
color task train accuracy: 89.3353941267388
shape task train accuracy: 98.96907216494846
opening task train accuracy: 33.8659793814433
strap task train accuracy: 85.16228748068006
pattern task train accuracy: 69.63917525773195
carrying task train accuracy: 99.69024264326278
charm task train accuracy: 99.38144329896907
decoration task train accuracy: 99.27872230808862


Total loss: 2.3165538. L2-loss: 1.7222764
Color loss: 0.10807786
Shape loss: 0.10745406
opening loss: 0.00020978256
strap loss: 0.07135235
pattern loss: 0.03169253
carrying loss: 0.10438572
charm loss: 0.10483197
decoration loss: 0.06627311


Epoch: 236
Learning rate: 0.000566
color task train accuracy: 89.43843379701185
shape task train accuracy: 98.81443298969073
opening task train accuracy: 33.83187209902012
strap task train accuracy: 85.25012893243941
pattern task train accuracy: 68.00618238021639
carrying task train accuracy: 99.38144329896907
charm task train accuracy: 99.48480164

Epoch: 248
Learning rate: 0.000485
color task train accuracy: 89.74226804123711
shape task train accuracy: 99.69072164948454
opening task train accuracy: 33.83187209902012
strap task train accuracy: 85.36082474226804
pattern task train accuracy: 67.42268041237114
carrying task train accuracy: 99.94845360824742
charm task train accuracy: 99.84536082474227
decoration task train accuracy: 99.53632148377125


Total loss: 1.8677723. L2-loss: 1.5722206
Color loss: 0.083016135
Shape loss: 0.0586937
opening loss: 0.00020998811
strap loss: 0.022133026
pattern loss: 0.02572915
carrying loss: 0.031147499
charm loss: 0.031099927
decoration loss: 0.043522414


Epoch: 249
Learning rate: 0.000461
color task train accuracy: 90.05154639175258
shape task train accuracy: 99.27797833935018
opening task train accuracy: 33.84853168469861
strap task train accuracy: 85.61855670103093
pattern task train accuracy: 69.21093347086126
carrying task train accuracy: 99.9484270242393
charm task train accuracy: 99.793

Epoch: 261
Learning rate: 0.000395
color task train accuracy: 90.00515198351366
shape task train accuracy: 99.63936115404431
opening task train accuracy: 33.83187209902012
strap task train accuracy: 85.61855670103093
pattern task train accuracy: 68.74677668901495
carrying task train accuracy: 99.89690721649484
charm task train accuracy: 99.94845360824742
decoration task train accuracy: 99.63917525773196


Total loss: 1.726303. L2-loss: 1.4191
Color loss: 0.061225515
Shape loss: 0.067610174
opening loss: 0.00021013212
strap loss: 0.031789493
pattern loss: 0.021741573
carrying loss: 0.045561567
charm loss: 0.04855581
decoration loss: 0.030508816


Epoch: 262
Learning rate: 0.000395
color task train accuracy: 90.0
shape task train accuracy: 99.27872230808862
opening task train accuracy: 33.883445074780816
strap task train accuracy: 85.41237113402062
pattern task train accuracy: 65.77319587628865
carrying task train accuracy: 99.63898916967509
charm task train accuracy: 99.63936115404431
d

Epoch: 274
Learning rate: 0.000339
color task train accuracy: 90.15463917525773
shape task train accuracy: 99.53560371517027
opening task train accuracy: 33.83187209902012
strap task train accuracy: 85.57444616177227
pattern task train accuracy: 70.06697578567749
carrying task train accuracy: 99.84536082474227
charm task train accuracy: 99.94845360824742
decoration task train accuracy: 99.69088098918083


Total loss: 1.5632612. L2-loss: 1.3093246
Color loss: 0.06337559
Shape loss: 0.04159473
opening loss: 0.00021310049
strap loss: 0.023038855
pattern loss: 0.022862023
carrying loss: 0.03227943
charm loss: 0.034900796
decoration loss: 0.035671987


Epoch: 275
Learning rate: 0.000339
color task train accuracy: 90.29927760577915
shape task train accuracy: 99.5360824742268
opening task train accuracy: 33.8659793814433
strap task train accuracy: 85.47140649149922
pattern task train accuracy: 67.26804123711341
carrying task train accuracy: 99.74240082431737
charm task train accuracy: 99.8968

Epoch: 287
Learning rate: 0.000290
color task train accuracy: 90.1081916537867
shape task train accuracy: 99.74226804123711
opening task train accuracy: 33.84853168469861
strap task train accuracy: 85.57444616177227
pattern task train accuracy: 70.72164948453609
carrying task train accuracy: 99.94848016486347
charm task train accuracy: 100.1031991744066
decoration task train accuracy: 99.74200206398349


Total loss: 1.4336056. L2-loss: 1.1907645
Color loss: 0.08120937
Shape loss: 0.033791363
opening loss: 0.00020704838
strap loss: 0.014867537
pattern loss: 0.013961017
carrying loss: 0.027856922
charm loss: 0.02517376
decoration loss: 0.045774017


Epoch: 288
Learning rate: 0.000290
color task train accuracy: 90.36082474226804
shape task train accuracy: 99.79392065945389
opening task train accuracy: 33.8659793814433
strap task train accuracy: 85.61855670103093
pattern task train accuracy: 65.30927835051547
carrying task train accuracy: 99.94845360824742
charm task train accuracy: 100.0
